# Tutorial-1

This second exercise demonstrates how to create a set of HDF5eis files with multi-dimensional data and link them to a single master file. To begin, let's import some necessary packages for this tutorial and create a directory where we can write some data files

In [1]:
# Standard library imports
import io
import pathlib

# Third-party imports
import hdf5eis
import numpy as np
import obspy.clients.fdsn
import pandas as pd


OUTPUT_DIR = pathlib.Path("/home/malcolmw/scratch/hdf5eis_tutorial")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

First let's simulate writing ten seconds of DAS data to ten separate files. Each file will contain one second of raw DAS data sampled from 1000 channels at 4000 samples per second.

In [2]:
data = np.random.rand(1000, 4000)
start_time = pd.to_datetime("2022-01-01T00:00:00Z")
sampling_rate = 4000
tag = "DAS/raw"

for i in range(10):
    with hdf5eis.File(OUTPUT_DIR.joinpath(f"file-{i:02d}.hdf5"), mode="w") as out_file:
        out_file.timeseries.add(
            data,
            start_time,
            sampling_rate,
            tag=tag
        )
    start_time = start_time + pd.to_timedelta(data.shape[1]/sampling_rate, unit="S")
        

Now we can link the individual files to one master file.

In [3]:
src_files = sorted(OUTPUT_DIR.iterdir())
                   
with hdf5eis.File(OUTPUT_DIR.joinpath(f"master.hdf5"), mode="w") as out_file:
    for src_file in src_files:
        out_file.timeseries.link_tag(src_file, "DAS/raw")

All ten files are now accessible from the master file, and data can be seamlessly read across file boundaries. Let's read one second of data spanning a file boundary from channels 256 through 511.

In [4]:
start_time = "2022-01-01T00:00:01.5Z"
end_time   = "2022-01-01T00:00:02.5Z"

with hdf5eis.File(OUTPUT_DIR.joinpath(f"master.hdf5"), mode="r") as in_file:
    gather = in_file.timeseries["DAS/raw", 256: 512, start_time: end_time]
    
gather

{'DAS/raw': [<hdf5eis.gather.Gather at 0x7ff1b926b370>]}

It may by inefficient or otherwise intractable to write the data all at once. For example, one may wish to convert a 2D array of 3C geophone data from miniSEED files to HDF5eis format. In cases such as this, one can create and empty HDF5eis DataSet and write data to it in a piece-wise fashion.

Consider a 2D array of 3C geophones with 16 rows and 32 columns. The below simulates conversion from channel-day miniSEED files to HDF5eis.

In [5]:
def read_miniSEED(station, channel, start_time, end_time):
    """
    This function simply returns some random data, but in reality would need to
    retrieve the actual data from the appropriate miniSEED file.
    """
    return (np.random.rand(8640000))

In [6]:
with hdf5eis.File(OUTPUT_DIR.joinpath("Array2D.hdf5"), mode="w") as out_file:
    ds = out_file.timeseries.create_dataset(
        (16, 16, 3, 8640000), # One day of data from a 16 by 16 array of 3C geophones sampled at 100 sps.
        "2022-01-01T00:00:00Z",
        100,
        tag="geophones/2D"
    )
    for irow in range(16):
        for icol in range(16):
            station = f"R{irow:02d}{icol:02d}"
            print(f"Adding data for station {station}.", end="\r")
            for icomp in range(3):
                ds[irow, icol, icomp, :] = read_miniSEED(
                    station, 
                    icomp, 
                    "2022-01-01T00:00:00Z",
                    "2022-01-02T00:00:00Z"
                )

We can link this to the same master file as the DAS data.

In [7]:
with hdf5eis.File(OUTPUT_DIR.joinpath("master.hdf5"), mode="a") as out_file:
    out_file.timeseries.link_tag(
        OUTPUT_DIR.joinpath("Array2D.hdf5"), 
        "geophones/2D"
    )

And we can slice across any of the storage dimensions. For example, let's slice on hour of data from rows 8 through 11, columns 2 through 9, and channels 0 and 1.

In [8]:
start_time = "2022-01-01T06:09:12.03Z"
end_time   = "2022-01-01T07:09:12.03Z"
with hdf5eis.File(OUTPUT_DIR.joinpath("master.hdf5"), mode="r") as in_file:
    gather = in_file.timeseries["geophones/2D", 8:12, 2: 10, :2, start_time: end_time]
    
gather["geophones/2D"][0].data.shape

(4, 8, 2, 360001)

Or we can extract one second of data from all channels.

In [9]:
start_time = "2022-01-01T06:09:12.03Z"
end_time   = "2022-01-01T06:09:13.03Z"
with hdf5eis.File(OUTPUT_DIR.joinpath("master.hdf5"), mode="r") as in_file:
    gather = in_file.timeseries["geophones/2D", ..., start_time: end_time]
    
gather["geophones/2D"][0].data.shape

(16, 16, 3, 101)

Those are the basics of ingesting and linking multi-dimensional data. For a more thorough example of ingesting data, see the `ingest_ZG_data` script for ingesting miniSEED data from the Sage Brush Flats Nodal Experiment (Ben-Zion et al., 2015).

Ben-Zion, Y., Vernon, F. L., Ozakin, Y., Zigone, D., Ross, Z. E., Meng, H., White, M., Reyes, J., Hollis, D., & Barklage, M. (2015). Basic data features  and results from a spatially dense seismic array on the San Jacinto fault zone. _Geophysical Journal International, 202_(1), 370–380. https://doi.org/10. 1093/gji/ggv142